In [223]:
import pandas as pd

In [224]:
df = pd.read_excel('base.xlsx') # processo base

In [225]:
# processos da minuta
minuta_geral = pd.read_excel('minuta.xlsx', sheet_name='GERAL')
minuta_credito = pd.read_excel('minuta.xlsx', sheet_name='CRÉDITO')
minuta_indenizatorios = pd.read_excel('minuta.xlsx', sheet_name='INDENIZATÓRIOS')
minuta_passagemdiarias = pd.read_excel('minuta.xlsx', sheet_name='PASSAGENSDIÁRIAS')

In [226]:
df.head()

,REGISTRO,ENTRADA,MÊS,PROCESSO,INTERESSADO,TIPO DESPESA,DETALHAMENTO,FONTE,VALOR,OBSERVAÇÃO,GRUPO DE DESPESA,DELIBERAÇÃO,ATA,SAÍDA,Unnamed: 14
0,Letícia,11.0,Janeiro,E:05501.0000003447/2024,DER,Contratação,Contratação de serviço de seguro de vida col...,500,"R$ 1.010,40",A legislação exige a proteção dos estagiários ...,Outros,SEGURAR,NaN,NaN,NaN
1,Letícia,9.0,Janeiro,E:04105.0000002178/2024,AMGESP,Contratos,Termo Aditivo ao Contrato CPL/SETRAND-DER/AL N...,500/501,"\nR$ 30.392.123,82",Tem como objeto os serviços Contínuos de Cons...,Outros,Aprovado com condicionante,1,2025-01-16 00:00:00,NaN
2,Letícia,11.0,Janeiro,E:05501.0000006142/2024,DER,NaN,NaN,NaN,NaN,NaN,Outros,Aprovado com condicionante,1,2025-01-16 00:00:00,NaN
3,Letícia,11.0,Janeiro,E:05501.0000006141/2024,DER,Contratos,Sexto Termo Aditivo ao Contrato CPL/SETRAND-DE...,500/501,"R$ 35.607.149,96",Aprovação condicionada da PGE. - prazo de vigê...,Outros,Aprovado com condicionante,1,2025-01-16 00:00:00,NaN
4,Beatriz,13.0,Janeiro,E:01101.0000004456/2023,GABCIVIL,Cessão de servidor,Renovação da cessão da servidora Othelina Amál...,-,-,O Setor de Análise e Instrução Processual – SA...,Outros,Aprovado com condicionante,NaN,NaN,NaN


In [227]:
# Mapping months to their respective numerical values
month_mapping = {
    'Janeiro': 1, 'Fevereiro': 2, 'Março': 3, 'Abril': 4, 'Maio': 5, 
    'Junho': 6, 'Julho': 7, 'Agosto': 8, 'Setembro': 9, 'Outubro': 10, 
    'Novembro': 11, 'Dezembro': 12
}

# Applying the mapping to the 'MÊS' column
df['MÊS'] = df['MÊS'].map(month_mapping)


In [228]:
df["ENTRADA"] = pd.to_numeric(df["ENTRADA"], errors='coerce').fillna(1).astype(int)


In [229]:
df["MÊS"] = pd.to_numeric(df["MÊS"], errors='coerce').fillna(12).astype(int)


In [230]:
# Coletar as informacoes da coluna MÊS e ENTRADA, onde entrada refere-se ao DIA e o mês ao mês, o ano sempre será 2025, e coloque na coliuna chamada Data de Recebimento
df['Data de Recebimento'] = pd.to_datetime(df['MÊS'].astype(str) + ' ' + df['ENTRADA'].astype(str) + ' 2025', format='%m %d %Y')


In [231]:
# EM Data de Recebimento, retirar o horario, deixar apenas a data memso
df['Data de Recebimento'] = df['Data de Recebimento'].dt.date

In [232]:
def limpar_valor_decimal(valor):
    # Transforma em string
    valor = str(valor).strip()

    # Remove "R$" e espaços extras
    valor = valor.replace('R$', '').strip()

    # Se for vazio ou traço, retorna 0.00
    if valor in ['-', ' -', '', None]:
        return 0.00

    # Se tem vírgula, trata como decimal
    if ',' in valor:
        valor = valor.replace('.', '')       # remove pontos (milhar)
        valor = valor.replace(',', '.')      # troca vírgula por ponto decimal
    else:
        valor = valor.replace('.', '')       # remove pontos
        valor = valor + '.00'                # adiciona .00 no final

    try:
        return float(valor)
    except ValueError:
        return 0.00


df['VALOR'] = df['VALOR'].apply(limpar_valor_decimal)


In [233]:
def formatar_fonte(valor):
    partes = str(valor).split('/')
    partes = [p.strip() for p in partes if p.strip()]

    if not partes:
        return ''
    elif len(partes) == 1:
        return partes[0]
    elif len(partes) == 2:
        return f'{partes[0]} e {partes[1]}'
    else:
        return ', '.join(partes[:-1]) + f' e {partes[-1]}'

df['FONTE'] = df['FONTE'].apply(formatar_fonte)


In [234]:
# remover entrada e mês
df = df.drop(columns=['ENTRADA', 'MÊS', "Unnamed: 14"])

In [235]:
# Renomear colunas
df.rename(columns={
    'REGISTRO': 'Cadastrado Por',
    'PROCESSO': 'Nº do Processo',
    'TIPO DESPESA': 'Tipo de Despesa',
    'INTERESSADO': 'Órgão (UO)',
    'FONTE': 'Fonte de Recursos',
    'GRUPO DE DESPESA': 'Grupo de Despesas',
    'VALOR': 'Valor',
    'DETALHAMENTO': 'Objetivo',
    'OBSERVAÇÃO': 'Observação',
    'Data de Recebimento': 'Data de Recebimento',
    'DELIBERAÇÃO': 'Deliberação',
    'ATA': 'Nº ATA',
    'SAÍDA': 'Data de Publicação'
}, inplace=True)

# Reorganizar as colunas na ordem especificada
df = df[['Deliberação', 'Nº do Processo', 'Tipo de Despesa', 'Órgão (UO)', 'Fonte de Recursos', 
         'Grupo de Despesas', 'Valor', 'Objetivo', 'Observação', 'Data de Recebimento', 
         'Data de Publicação', 'Nº ATA', 'Cadastrado Por']]

In [236]:
df.head()

,Deliberação,Nº do Processo,Tipo de Despesa,Órgão (UO),Fonte de Recursos,Grupo de Despesas,Valor,Objetivo,Observação,Data de Recebimento,Data de Publicação,Nº ATA,Cadastrado Por
0,SEGURAR,E:05501.0000003447/2024,Contratação,DER,500,Outros,1010.40,Contratação de serviço de seguro de vida col...,A legislação exige a proteção dos estagiários ...,2025-01-11,NaN,NaN,Letícia
1,Aprovado com condicionante,E:04105.0000002178/2024,Contratos,AMGESP,500 e 501,Outros,30392123.82,Termo Aditivo ao Contrato CPL/SETRAND-DER/AL N...,Tem como objeto os serviços Contínuos de Cons...,2025-01-09,2025-01-16 00:00:00,1,Letícia
2,Aprovado com condicionante,E:05501.0000006142/2024,NaN,DER,nan,Outros,0.00,NaN,NaN,2025-01-11,2025-01-16 00:00:00,1,Letícia
3,Aprovado com condicionante,E:05501.0000006141/2024,Contratos,DER,500 e 501,Outros,35607149.96,Sexto Termo Aditivo ao Contrato CPL/SETRAND-DE...,Aprovação condicionada da PGE. - prazo de vigê...,2025-01-11,2025-01-16 00:00:00,1,Letícia
4,Aprovado com condicionante,E:01101.0000004456/2023,Cessão de servidor,GABCIVIL,-,Outros,0.00,Renovação da cessão da servidora Othelina Amál...,O Setor de Análise e Instrução Processual – SA...,2025-01-13,NaN,NaN,Beatriz


In [237]:
# Coluna Deliberação unique
df['Deliberação'].unique()

array(['SEGURAR', 'Aprovado com condicionante', 'Diligência', nan,
       'Aprovado', 'Indeferido', 'Devolvido', 'INTEMPESTIVO',
       'Material de consumo', 'Aprovado parcial', 'Vencido', 'VENCIDO'],
      dtype=object)

In [238]:
# Onde tiver VENCIDO colocar Vencido, onde tiver INTEMPESTIVO, colocar Intempestivo
df['Deliberação'] = df['Deliberação'].replace({'VENCIDO': 'Vencido', 'INTEMPESTIVO': 'Intempestivo', "SEGURAR": "Segurar"})

In [239]:
# verificar todo o df, o que for NaN, colocar como """", e as colunas de valores, colocar como 0.00
df = df.fillna({
    'Deliberação': 'Disponível aos Membros CPOF',
    'Nº do Processo': 'Sem Informação',
    'Tipo de Despesa': 'Sem Informação',
    'Órgão (UO)': 'Sem Informação',
    'Fonte de Recursos': 'Sem Informação',
    'Grupo de Despesas': 'Sem Informação',
    'Valor': 0.00,
    'Detalhamento': 'Sem Informação',
    'Observação': 'Sem Informação',
    'Data de Recebimento': pd.NaT,
    'Data de Publicação': pd.NaT,
    'ATA': "Sem Informação",
    'Cadastrado Por': 'Sem Informação'
})

In [240]:
df.head()

,Deliberação,Nº do Processo,Tipo de Despesa,Órgão (UO),Fonte de Recursos,Grupo de Despesas,Valor,Objetivo,Observação,Data de Recebimento,Data de Publicação,Nº ATA,Cadastrado Por
0,Segurar,E:05501.0000003447/2024,Contratação,DER,500,Outros,1010.40,Contratação de serviço de seguro de vida col...,A legislação exige a proteção dos estagiários ...,2025-01-11,NaT,NaN,Letícia
1,Aprovado com condicionante,E:04105.0000002178/2024,Contratos,AMGESP,500 e 501,Outros,30392123.82,Termo Aditivo ao Contrato CPL/SETRAND-DER/AL N...,Tem como objeto os serviços Contínuos de Cons...,2025-01-09,2025-01-16 00:00:00,1,Letícia
2,Aprovado com condicionante,E:05501.0000006142/2024,Sem Informação,DER,nan,Outros,0.00,NaN,Sem Informação,2025-01-11,2025-01-16 00:00:00,1,Letícia
3,Aprovado com condicionante,E:05501.0000006141/2024,Contratos,DER,500 e 501,Outros,35607149.96,Sexto Termo Aditivo ao Contrato CPL/SETRAND-DE...,Aprovação condicionada da PGE. - prazo de vigê...,2025-01-11,2025-01-16 00:00:00,1,Letícia
4,Aprovado com condicionante,E:01101.0000004456/2023,Cessão de servidor,GABCIVIL,-,Outros,0.00,Renovação da cessão da servidora Othelina Amál...,O Setor de Análise e Instrução Processual – SA...,2025-01-13,NaT,NaN,Beatriz


In [241]:
df["Grupo de Despesas"].unique()

array(['Outros', 'Investimento', 'Sem Informação'], dtype=object)

In [242]:
df.columns

Index(['Deliberação', 'Nº do Processo', 'Tipo de Despesa', 'Órgão (UO)',
       'Fonte de Recursos', 'Grupo de Despesas', 'Valor', 'Objetivo',
       'Observação', 'Data de Recebimento', 'Data de Publicação', 'Nº ATA',
       'Cadastrado Por'],
      dtype='object')

In [243]:
# Adicione colunas e deixe-as vazias
# Última Edição, SECRETÁRIA EXECUTIVA, SEPLAG, SEFAZ, GABINETE CIVIL, SEGOV
df['Última Edição'] = ''
df['SECRETÁRIA EXECUTIVA'] = ''
df['SEPLAG'] = ''
df['SEFAZ'] = ''
df['GABINETE CIVIL'] = ''
df['SEGOV'] = ''

In [244]:
df.head()

,Deliberação,Nº do Processo,Tipo de Despesa,Órgão (UO),Fonte de Recursos,Grupo de Despesas,Valor,Objetivo,Observação,Data de Recebimento,Data de Publicação,Nº ATA,Cadastrado Por,Última Edição,SECRETÁRIA EXECUTIVA,SEPLAG,SEFAZ,GABINETE CIVIL,SEGOV
0,Segurar,E:05501.0000003447/2024,Contratação,DER,500,Outros,1010.40,Contratação de serviço de seguro de vida col...,A legislação exige a proteção dos estagiários ...,2025-01-11,NaT,NaN,Letícia,,,,,,
1,Aprovado com condicionante,E:04105.0000002178/2024,Contratos,AMGESP,500 e 501,Outros,30392123.82,Termo Aditivo ao Contrato CPL/SETRAND-DER/AL N...,Tem como objeto os serviços Contínuos de Cons...,2025-01-09,2025-01-16 00:00:00,1,Letícia,,,,,,
2,Aprovado com condicionante,E:05501.0000006142/2024,Sem Informação,DER,nan,Outros,0.00,NaN,Sem Informação,2025-01-11,2025-01-16 00:00:00,1,Letícia,,,,,,
3,Aprovado com condicionante,E:05501.0000006141/2024,Contratos,DER,500 e 501,Outros,35607149.96,Sexto Termo Aditivo ao Contrato CPL/SETRAND-DE...,Aprovação condicionada da PGE. - prazo de vigê...,2025-01-11,2025-01-16 00:00:00,1,Letícia,,,,,,
4,Aprovado com condicionante,E:01101.0000004456/2023,Cessão de servidor,GABCIVIL,-,Outros,0.00,Renovação da cessão da servidora Othelina Amál...,O Setor de Análise e Instrução Processual – SA...,2025-01-13,NaT,NaN,Beatriz,,,,,,


In [245]:
minuta_geral.head()

,PROCESSO,INTERESSADO,TIPO DE DESPESA,DETALHAMENTO,FONTE,VALOR,SEX. EXECUTIVA,SEPLAG,SEFAZ,GAB. CIVIL,SEGOV,OBSERVAÇÃO
0,E:01203.0000002914/2025,CBMAL,Aquisição,Aquisição de Cilindro de Ar Comprimido (20).,759/753,"R$ 44.580,00","Aprovar, condicionado as fontes 759 e 753",Segue SE,SEGUE SE,NaN,NaN,Necessidade de uso em ocorrências de resgate e...
1,E:01203.0000003182/2025,CBMAL,Aquisição,Aquisição de 2 Computadores de Mergulho.,501/759,15000,Aprovar,Segue SE,SEGUE SE,NaN,NaN,O referido equipamento será destinado à guarni...
2,E:01203.0000003408/2025,CBMAL,Aquisição,Aquisição de 02 viaturas administrativas.,713,"R$ 516.302,00","Aprovar, condicionado a fonte 713",Segue SE,SEGUE SE,NaN,NaN,DILIGÊNCIA NA 28ª ATA: Informaram que para pro...
3,E:05101.0000001560/2025,DETRAN,Aquisição,Aquisição de 2500 placas para piso elevado pa...,501,"R$ 480.075,00",Quando foi instalado o piso atual? Essa quanti...,Deve constar nos autos o registro fotográfico ...,SEGUE SE E SEPLAG,NaN,NaN,Aquisição de placas para piso elevado pela nec...
4,E:05101.0000012249/2024,DETRAN,Contratos,Contratação EMERGENCIAL do serviço especializ...,752,"R$ 3.769.440,00",Aprovar condicionado ao andamento do processo ...,Segue SE,sEGUE SE,NaN,NaN,Se encontra em tramitação o processo E:05101.0...


In [246]:
import pandas as pd

# Leitura dos arquivos
minuta_geral = pd.read_excel('minuta.xlsx', sheet_name='GERAL')
minuta_credito = pd.read_excel('minuta.xlsx', sheet_name='CRÉDITO')
minuta_indenizatorios = pd.read_excel('minuta.xlsx', sheet_name='INDENIZATÓRIOS')
minuta_passagemdiarias = pd.read_excel('minuta.xlsx', sheet_name='PASSAGENSDIÁRIAS')

# Função para buscar e preencher os dados das colunas específicas
def atualizar_colunas_por_processo(base_df, ref_df, col_processo_base, col_processo_ref, colunas_origem, colunas_destino):
    for i, processo in base_df[col_processo_base].items():
        if pd.isna(processo):
            continue
        linha_ref = ref_df[ref_df[col_processo_ref].astype(str).str.contains(str(processo), na=False)]
        if not linha_ref.empty:
            for col_origem, col_destino in zip(colunas_origem, colunas_destino):
                base_df.at[i, col_destino] = linha_ref.iloc[0][col_origem]
    return base_df

# Define colunas a serem transferidas
colunas_origem = ['SEX. EXECUTIVA', 'SEPLAG', 'SEFAZ', 'GAB. CIVIL', 'SEGOV']
colunas_destino = ['SECRETÁRIA EXECUTIVA', 'SEPLAG', 'SEFAZ', 'GABINETE CIVIL', 'SEGOV']

# Atualiza df com base nas outras planilhas
df = atualizar_colunas_por_processo(
    base_df=df,
    ref_df=minuta_credito,
    col_processo_base='Nº do Processo',
    col_processo_ref='PROCESSO',
    colunas_origem=colunas_origem,
    colunas_destino=colunas_destino
)

df = atualizar_colunas_por_processo(
    base_df=df,
    ref_df=minuta_indenizatorios,
    col_processo_base='Nº do Processo',
    col_processo_ref='PROCESSO',
    colunas_origem=colunas_origem,
    colunas_destino=colunas_destino
)

df = atualizar_colunas_por_processo(
    base_df=df,
    ref_df=minuta_passagemdiarias,
    col_processo_base='Nº do Processo',
    col_processo_ref='PROCESSO',
    colunas_origem=colunas_origem,
    colunas_destino=colunas_destino
)


In [247]:
df.head()

,Deliberação,Nº do Processo,Tipo de Despesa,Órgão (UO),Fonte de Recursos,Grupo de Despesas,Valor,Objetivo,Observação,Data de Recebimento,Data de Publicação,Nº ATA,Cadastrado Por,Última Edição,SECRETÁRIA EXECUTIVA,SEPLAG,SEFAZ,GABINETE CIVIL,SEGOV
0,Segurar,E:05501.0000003447/2024,Contratação,DER,500,Outros,1010.40,Contratação de serviço de seguro de vida col...,A legislação exige a proteção dos estagiários ...,2025-01-11,NaT,NaN,Letícia,,,,,,
1,Aprovado com condicionante,E:04105.0000002178/2024,Contratos,AMGESP,500 e 501,Outros,30392123.82,Termo Aditivo ao Contrato CPL/SETRAND-DER/AL N...,Tem como objeto os serviços Contínuos de Cons...,2025-01-09,2025-01-16 00:00:00,1,Letícia,,,,,,
2,Aprovado com condicionante,E:05501.0000006142/2024,Sem Informação,DER,nan,Outros,0.00,NaN,Sem Informação,2025-01-11,2025-01-16 00:00:00,1,Letícia,,,,,,
3,Aprovado com condicionante,E:05501.0000006141/2024,Contratos,DER,500 e 501,Outros,35607149.96,Sexto Termo Aditivo ao Contrato CPL/SETRAND-DE...,Aprovação condicionada da PGE. - prazo de vigê...,2025-01-11,2025-01-16 00:00:00,1,Letícia,,,,,,
4,Aprovado com condicionante,E:01101.0000004456/2023,Cessão de servidor,GABCIVIL,-,Outros,0.00,Renovação da cessão da servidora Othelina Amál...,O Setor de Análise e Instrução Processual – SA...,2025-01-13,NaT,NaN,Beatriz,,,,,,


In [248]:
df.dtypes

Deliberação              object
Nº do Processo           object
Tipo de Despesa          object
Órgão (UO)               object
Fonte de Recursos        object
Grupo de Despesas        object
Valor                   float64
Objetivo                 object
Observação               object
Data de Recebimento      object
Data de Publicação       object
Nº ATA                   object
Cadastrado Por           object
Última Edição            object
SECRETÁRIA EXECUTIVA     object
SEPLAG                   object
SEFAZ                    object
GABINETE CIVIL           object
SEGOV                    object
dtype: object

In [249]:
df['Órgão (UO)'] = df['Órgão (UO)'].str.strip()
df['Órgão (UO)'] = df['Órgão (UO)'].str.upper()

In [250]:
# AL PREV -> ALPREV
df['Órgão (UO)'] = df['Órgão (UO)'].replace({'AL PREV': 'ALPREV'})


In [251]:
df['Órgão (UO)'].unique()

array(['DER', 'AMGESP', 'GABCIVIL', 'SECTI', 'SEFAZ', 'SECDEF', 'POLCAL',
       'ADEAL', 'SETUR', 'SEAGRI', 'VICEGOV', 'JUCEAL', 'UNCISAL',
       'SEPLAG', 'SECULT', 'SEDICS', 'SETRAND', 'CBMAL', 'SESAU',
       'IDERAL', 'SEINFRA', 'ALPREV', 'SETEQ', 'PMAL', 'PGE', 'SECOM',
       'SEADES', 'SERIS', 'SSP', 'PCAL', 'DETRAN', 'SEMUDH', 'SEDUC',
       'UNEAL', 'SELAJ', 'SECRIA', 'FAPEAL', 'SERFI', 'IPASEAL', 'SEMARH',
       'SEPREV', 'ITERAL', 'EMATER', 'CGE', 'ARSAL', 'CARHP', 'LIFAL',
       'DPEAL', 'SEGOV', 'PROCON', 'IMA', 'IZP', 'ITEC', 'DEA', 'CEDEC'],
      dtype=object)

In [252]:
# em Fonte de Recursos, o que for missin value, ou "-", colocar 500
df['Fonte de Recursos'] = df['Fonte de Recursos'].replace({'-': 500, pd.NA: 500})

In [253]:

df['Fonte de Recursos'] = df['Fonte de Recursos'].fillna(500)

In [254]:
df['Fonte de Recursos'] = df['Fonte de Recursos'].replace({'nan': 500})

In [255]:
df['Grupo de Despesas'] = df['Grupo de Despesas'].replace({'Investimento':4})
df['Grupo de Despesas'] = df['Grupo de Despesas'].replace({'Sem Informação':'Outros'})

In [256]:
df.head()

,Deliberação,Nº do Processo,Tipo de Despesa,Órgão (UO),Fonte de Recursos,Grupo de Despesas,Valor,Objetivo,Observação,Data de Recebimento,Data de Publicação,Nº ATA,Cadastrado Por,Última Edição,SECRETÁRIA EXECUTIVA,SEPLAG,SEFAZ,GABINETE CIVIL,SEGOV
0,Segurar,E:05501.0000003447/2024,Contratação,DER,500,Outros,1010.40,Contratação de serviço de seguro de vida col...,A legislação exige a proteção dos estagiários ...,2025-01-11,NaT,NaN,Letícia,,,,,,
1,Aprovado com condicionante,E:04105.0000002178/2024,Contratos,AMGESP,500 e 501,Outros,30392123.82,Termo Aditivo ao Contrato CPL/SETRAND-DER/AL N...,Tem como objeto os serviços Contínuos de Cons...,2025-01-09,2025-01-16 00:00:00,1,Letícia,,,,,,
2,Aprovado com condicionante,E:05501.0000006142/2024,Sem Informação,DER,500,Outros,0.00,NaN,Sem Informação,2025-01-11,2025-01-16 00:00:00,1,Letícia,,,,,,
3,Aprovado com condicionante,E:05501.0000006141/2024,Contratos,DER,500 e 501,Outros,35607149.96,Sexto Termo Aditivo ao Contrato CPL/SETRAND-DE...,Aprovação condicionada da PGE. - prazo de vigê...,2025-01-11,2025-01-16 00:00:00,1,Letícia,,,,,,
4,Aprovado com condicionante,E:01101.0000004456/2023,Cessão de servidor,GABCIVIL,500,Outros,0.00,Renovação da cessão da servidora Othelina Amál...,O Setor de Análise e Instrução Processual – SA...,2025-01-13,NaT,NaN,Beatriz,,,,,,


In [257]:
df['Grupo de Despesas'].unique()

array(['Outros', 4], dtype=object)

In [258]:
# Na coluna Tipo de Despesa, o que for Sem Informação, colocar Outros
df['Tipo de Despesa'] = df['Tipo de Despesa'].replace({'Sem Informação': 'Outros'})

In [259]:
df['Tipo de Despesa'].unique()

array(['Contratação', 'Contratos', 'Outros', 'Cessão de servidor',
       'Parceria', 'Fomento', 'Aquisição', 'Diária', 'Passagem',
       'Patrocínio', 'Crédito', 'Capacitação', 'Chamamento público',
       'Locação de Veículo', 'Locação de Imóvel', 'Convênio'],
      dtype=object)

In [260]:
df.head()

,Deliberação,Nº do Processo,Tipo de Despesa,Órgão (UO),Fonte de Recursos,Grupo de Despesas,Valor,Objetivo,Observação,Data de Recebimento,Data de Publicação,Nº ATA,Cadastrado Por,Última Edição,SECRETÁRIA EXECUTIVA,SEPLAG,SEFAZ,GABINETE CIVIL,SEGOV
0,Segurar,E:05501.0000003447/2024,Contratação,DER,500,Outros,1010.40,Contratação de serviço de seguro de vida col...,A legislação exige a proteção dos estagiários ...,2025-01-11,NaT,NaN,Letícia,,,,,,
1,Aprovado com condicionante,E:04105.0000002178/2024,Contratos,AMGESP,500 e 501,Outros,30392123.82,Termo Aditivo ao Contrato CPL/SETRAND-DER/AL N...,Tem como objeto os serviços Contínuos de Cons...,2025-01-09,2025-01-16 00:00:00,1,Letícia,,,,,,
2,Aprovado com condicionante,E:05501.0000006142/2024,Outros,DER,500,Outros,0.00,NaN,Sem Informação,2025-01-11,2025-01-16 00:00:00,1,Letícia,,,,,,
3,Aprovado com condicionante,E:05501.0000006141/2024,Contratos,DER,500 e 501,Outros,35607149.96,Sexto Termo Aditivo ao Contrato CPL/SETRAND-DE...,Aprovação condicionada da PGE. - prazo de vigê...,2025-01-11,2025-01-16 00:00:00,1,Letícia,,,,,,
4,Aprovado com condicionante,E:01101.0000004456/2023,Cessão de servidor,GABCIVIL,500,Outros,0.00,Renovação da cessão da servidora Othelina Amál...,O Setor de Análise e Instrução Processual – SA...,2025-01-13,NaT,NaN,Beatriz,,,,,,


In [261]:

df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], errors='coerce').dt.date

In [262]:
df.to_excel('base_filtrada.xlsx', index=False)

In [263]:
# df_final = pd.read_excel('base_filtrada.xlsx')

In [264]:
# df_final.head()